In [1]:
!pip install ultralytics

     -------------------------------------- 645.7/645.7 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 6.0 MB/s eta 0:00:00
     -------------------------------------- 192.2/192.2 MB 3.6 MB/s eta 0:00:00
     ------------------------------------- 166.4/166.4 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1


In [2]:
from ultralytics import YOLO
model_path = "best.pt"  # Replace with the path to your YOLOv8 model file
model = YOLO(model_path)
model.load()

Transferred 319/355 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [3]:
#using the results we are drawing bounding boxes 
import cv2
import torch

cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Convert the frame to a PyTorch tensor
    img = torch.from_numpy(frame).unsqueeze(0).permute(0, 3, 1, 2).float() / 255.0

    # Make predictions
    results = model(img)
    # Process and draw bounding boxes on the frame
    for det in results:  # Access bounding box information
        bbox = det.boxes.xyxy.tolist()
        if len(bbox) != 0:
            x_min, y_min, x_max, y_max = bbox[0][:4]
            x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
            if det.boxes[0].conf <0.3:
                label = model.names[int(det.boxes[0].cls)]
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, f"Fall: {det.boxes[0].conf.item():.2f}", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            else:
                label = model.names[int(det.boxes[0].cls)]
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
                cv2.putText(frame, f"Not Fall: {det.boxes[0].conf.item():.2f}", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    # Display the frame
    cv2.imshow("YOLO Live Video", frame)

        # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 non_fall, 158.1ms
Speed: 1.0ms preprocess, 158.1ms inference, 13.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 121.2ms
Speed: 0.0ms preprocess, 121.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 79.1ms
Speed: 0.0ms preprocess, 79.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 78.1ms
Speed: 0.0ms preprocess, 78.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 82.0ms
Speed: 0.0ms preprocess, 82.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 79.0ms
Speed: 0.0ms preprocess, 79.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 75.3ms
Speed: 0.0ms preprocess, 75.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 81.7ms
Speed: 0.0ms preprocess, 81.7ms i

TypeError: expected np.ndarray (got NoneType)

: 

In [ ]:
# this code directly draws boundin boxes and displays it, output object(results) takes care of everything 
import cv2
import torch
cap = cv2.VideoCapture(0)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True,conf=0.5)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()
    
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
